# Imports 

In [2]:
!pip install idx2numpy
!pip install python_mnist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-cp37-none-any.whl size=7907 sha256=3e196d217dcba7e7c83cd942bc44cce217b0c98741386e8e7bae520a8d5390c7
  Stored in directory: /root/.cache/pip/wheels/7a/c1/da/284ce80a748fab898b8d1fa95468a386e7cf3b81da18511f9d
Successfully built idx2numpy
Mounted at /content/drive


#Setup

In [3]:
train_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/datasets/train-gaussian.csv")
test_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/datasets/test-gaussian.csv")
print(train_data.shape)
print(test_data.shape)

(1891, 4)
(830, 4)


In [4]:
x_train = train_data[[' x1',' x2',' x3']].values
y_train = train_data['class'].values
x_test = test_data[['x1','x2','x3']].values
y_test = test_data['class'].values
classes = ['A','B']

In [5]:
print(np.mean(x_train[y_train=='A'],axis=0))
print(len(x_train[y_train=='A']))
print(np.mean(x_train[y_train=='B'],axis=0))
print(len(x_train[y_train=='B']))

[-10.56044211  -3.63296938  -8.32880383]
1045
[-13.52702955  -4.41293026  -4.52632388]
846


# Question 3.1

In [93]:
classmeans = []
cov = np.cov(x_train.T)
diagcov = np.diagflat(np.diag(cov))
currclass = 0
for c in classes:
  mean = np.mean(x_train[y_train==c],axis=0)
  classmeans.append(mean)
acc = 0
diagacc=0
for x in range(len(x_test)):
  A = np.exp( -((x_test[x]-classmeans[0]).dot( np.linalg.inv(cov))).dot((x_test[x]-classmeans[0])) /2)/ ((2*np.pi )**(3/2))*(np.sqrt(np.linalg.det(cov)))
  B = np.exp( -((x_test[x]-classmeans[1]).dot( np.linalg.inv(cov))).dot((x_test[x]-classmeans[1])) /2)/ ((2*np.pi )**(3/2))*(np.sqrt(np.linalg.det(cov)))
  if classes[np.argmax((A,B))] == y_test[x]:
    acc +=1
  diagA = np.exp( -((x_test[x]-classmeans[0]).dot( np.linalg.inv(diagcov))).dot((x_test[x]-classmeans[0])) /2)/ ((2*np.pi )**(3/2))*(np.sqrt(np.linalg.det(diagcov)))
  diagB = np.exp( -((x_test[x]-classmeans[1]).dot( np.linalg.inv(diagcov))).dot((x_test[x]-classmeans[1])) /2)/ ((2*np.pi )**(3/2))*(np.sqrt(np.linalg.det(diagcov)))
  if classes[np.argmax((diagA,diagB))] == y_test[x]:
    diagacc +=1
print("Full covariance accuracy: {:.5f}%".format(acc/x_test.shape[0]*100))
print("Diagonal covariance accuracy: {:.5f}%".format(diagacc/x_test.shape[0]*100))

Full covariance accuracy: 73.37349%
Diagonal covariance accuracy: 74.09639%


# Question 3.2

In [112]:
#K means clustering method to initialize
m= 8
traindata = ( [ (t[0],t[1:]) for t in train_data.values])
x_data =  [ t[1:] for t in train_data.values]
k=0
clusters = []
clusters.append([None] for i in range(m-1))
centroids = [np.mean([t[1] for t in traindata],axis=0),(0 for i in range(m-1))]
while k < m:
  continu = True
  while continu:
    centroidcopy = [0 for i in range(m)]
    clust = [[] for i in range(m)]
    for xy in traindata:
      x = xy[1]
      meandiff = [  np.linalg.norm(x-centroids[i]) for i in range(k+1) ]
      closest = np.argmin(meandiff)
      clust[closest].append(xy)
    for i in range(k+1):
      centroidcopy[i] = (np.mean([t[1] for t in clust[i]],axis=0))
    continu = False
    for i in range(k):
      continu = True if (centroids[i] != centroidcopy[i]).any() else False
    centroids = centroidcopy
    clusters=clust
  if k+1 < m:
    splitpos = np.argmax([ len(t) for t in clusters])
    secondhalf = clusters[splitpos][int(np.round(len(clusters[splitpos])/2)):]
    clusters[splitpos] = clusters[splitpos][:int(np.round(len(clusters[splitpos])/2))]
    centroids[splitpos] = (np.mean([t[1] for t in clusters[splitpos]]))
    clusters[k+1] = secondhalf
    centroids[k+1] = (np.mean([t[1] for t in clusters[k+1]]))
  print(k)
  k+=1

weights = []
means = []
covariances = []
for cluster in clusters:
  weights.append((len(cluster)/len(traindata)))
  means.append(np.mean(np.asarray(([t[1] for t in cluster])).astype("float"),axis=0).reshape(1,3))
  cov = np.asarray([t[1] for t in cluster]).astype("float")
  covariances.append(np.cov(cov.T))
weights = np.asarray( weights)
means = np.asarray(means)

0
1
2
3
4
5
6
7


In [108]:
#for using diagonal covariances
covariances = [np.diagflat(np.diag(cov)) for cov in covariances]

In [114]:
#EM GMM
Em = [[] for i in range(m)]
cont = True
l = 0
llikelihood = [1e10]
while cont:
  tmpweights = np.ones(weights.shape)
  for x in train_data.values:
    x=x[1:]
    gauss = [(np.exp( (-((x-means[a]).dot( np.linalg.inv(covariances[a]))).dot((x-means[a]).T) /2).astype("float"))\
    / ((2*np.pi )**(3/2))*(np.sqrt(np.abs(np.linalg.det(covariances[a]))))) for a in range(m)]
    for i in range(m):
      Em[i].append((weights[i] * (gauss[i] ) ) / (np.sum(weights*gauss) ))
  for i in range(m):
    means[i] = np.sum([Em[i][j]*x_data[j] for j in range(len(x_data))] )/np.sum(Em[i])
    covariances[i] = np.sum([ (Em[i][j])* (x_data[j]-means[i]).T*(x_data[j]-means[i]) for j in range(len(x_data))  ],axis=0).astype("float") /np.sum(Em[i])
    tmpweights[i] = np.sum(Em[i])/len(x_data)
  if (weights == tmpweights).all():
    cont = False
  weights = tmpweights
  l+=1
  ll = 0
  for E in Em:
    ll +=  np.abs(np.mean(np.log([E[i] for i in range(len(E))])))
  llikelihood.append(ll)
  print(l, ll)
  if (llikelihood[-2] - llikelihood[-1]) < 1e-1:
    cont = False

1 125.90288797986582
2 103.7067559492977
3 110.30924873028864


In [115]:
print(weights)
print(means)
print(covariances)
#### Found that 8 clusters and diagonal covariances were the best setup

[0.01848618 0.12238092 0.00485716 0.02983446 0.03308228 0.05004944
 0.08595628 0.05480024]
[[[-16.54055392 -16.54055392 -16.54055392]]

 [[ -2.7592147   -2.7592147   -2.7592147 ]]

 [[-14.49184863 -14.49184863 -14.49184863]]

 [[ -9.84079784  -9.84079784  -9.84079784]]

 [[-12.33102346 -12.33102346 -12.33102346]]

 [[ -4.94369843  -4.94369843  -4.94369843]]

 [[ -7.84748333  -7.84748333  -7.84748333]]

 [[ -8.33001902  -8.33001902  -8.33001902]]]
[array([[27.75924733, 14.42248453, -5.80758392],
       [14.42248453, 14.6355803 , -3.58561532],
       [-5.80758392, -3.58561532,  5.92572906]]), array([[ 29.93713933,  -4.98540276, -10.00074506],
       [ -4.98540276,   2.36950921,   1.802319  ],
       [-10.00074506,   1.802319  ,   4.57146344]]), array([[  6.25501235,  23.99660001,  -4.0023933 ],
       [ 23.99660001, 141.23477599, -30.60084269],
       [ -4.0023933 , -30.60084269,  11.87772482]]), array([[ 4.09659094,  7.68626254, -0.44175709],
       [ 7.68626254, 27.5610144 , -1.3539916